<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/tabular_neural_net_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Model For Waste Data Classification


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.
df - data_frame

In [4]:
# mount to google drive and load files
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# check file is loaded OK
path = '/content/drive/My Drive/data/'
print(path)

/content/drive/My Drive/data/


In [6]:
# load train data frame
# todo: 1. iterate through nl data and export only with material, eural code fields
#  then translate to en. this will increase train and test data sets and include missing material categories that appear in validation set
df = pd.read_csv(path + '/minimizedData.csv')



# load test data frame with material field empty
test_df = pd.read_csv(path + '/testNoMaterialData.csv')

df.head()

,reason,origin,color,state,size,consistency,otherCode,material4,material3,material2,material,mType,composite2,composite1,cType,indirectProduct,directProduct,pType,mixedOrPure,cleanOrDirty,euralDescription,euralCode,description,/0
0,NaN,NaN,NaN,NaN,NaN,slurry,NaN,NaN,NaN,oil,organic material,Soy,NaN,dry,NaN,dry,NaN,NaN,1.0,NaN,material unsuitable for consumption or processing,20304,Soyadroes technically,NaN
1,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,wood,branches,NaN,NaN,NaN,wood,NaN,NaN,0.0,NaN,waste from forestry,20107,Branches,NaN
2,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,cocoa,caps,NaN,NaN,NaN,cocoa,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,Cocoa shells,NaN
3,NaN,NaN,NaN,debris,NaN,vast,NaN,NaN,NaN,NaN,sand,NaN,NaN,debris,NaN,debris,NaN,NaN,1.0,1.0,"mixtures of concrete, stones, tiles or ceramic...",170107,Debris with Sand,NaN
4,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,organic material,Soy,NaN,NaN,NaN,organic material,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,soya,NaN


#cats and conts 

**categorical variables - cat_names:**

what you're using to make predictions with.  (description, euralDescription, composite1, mType).
for categorical variables - will use Embeddings 

**continuos variables - cont_names:**

any catgeory that has limited number of possible (integer) choises: true/false ; fields such as cleanOrContaminated or mixedOrPure
continuos varibale names - they are sent to Neural net as pixels 

**uses processor - procs:**

a number of processes that deal with data in some ways: FillMissing, Categorify, Normalize both train and test dtata sets

In [0]:
# dependable variable field:
dep_var = 'material'
 
# can add composite1, mType
cat_names = ['euralDescription', 'description', 'euralCode']
cont_names = []
# cont_names = ['mixedOrPure', 'cleanOrDirty']
# processors:
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[950:1200].copy(), path=path + '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names)

Next:
data block api
add labels - tell which cols - column it is 

and get back data bunch
split train and test data ranges 800,1000


In [9]:
data = (TabularList.from_df(df, path=path+ '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(950,1200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
non-iron, PCFIC, syrup, zeolite, solvents...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [10]:
data.show_batch(rows=10)

euralDescription,description,euralCode,target
discarded electrical and electronic equipment other than those mentioned in 20 01 21 and 20 01 23 containing dangerous components (6),Objects with Beryllium,200135,beryllium
building material containing asbestos,9401 rubble containing asbestos,170605,asbestos
soil and stones not covered by 17 05 03,"Soil, contaminated",170504,ground
biodegradable waste,planting waste / logs,200201,organic material
biodegradable waste,5230 garden dirt chopped,200201,organic material
biodegradable waste,"Agricultural, clean",200201,organic material
cooking oil and fats,oils and fats,200125,know
soil and stones not covered by 17 05 03,Ground AP04 AW,170504,ground
soil and stones not covered by 17 05 03,slightly troubled,170504,ground
biodegradable waste,GFT Cocoa powder,200201,organic material


In [14]:
print(len(data.classes), 'categories for material --->', data.classes, )

95 categories for material ---> ['AVI-bodemas', 'Cork', 'EPS', 'Glass', 'HDPE', 'LDPE', 'ON', 'PA', 'PC', 'PET', 'PFOS', 'PMMA', 'POM', 'PP', 'PS', 'PVC', 'Perlite', 'RIGHT NOW', 'RVS', 'SECTION', 'Soy', 'UP', 'activated carbon', 'aluminum', 'asbestos', 'beer', 'believe', 'bentonite', 'beryllium', 'bitumen', 'bold', 'bread', 'carton', 'cast', 'cement', 'ceramic', 'ceramic material', 'cocoa', 'collect', 'compost', 'dill', 'dough', 'eps', 'fatty acids', 'fly ash', 'foam rubber', 'glass wool', 'glycerine', 'grind', 'ground', 'insulation material', 'iron', 'know', 'kraftkarton', 'kwik', 'lavaliet', 'lime', 'lithium', 'magnesium', 'melamine', 'messing', 'metal', 'minerals', 'molasses', 'most', 'nickel', 'nylon', 'oil', 'optical fiber', 'organic material', 'paper', 'plastic', 'polyester', 'porcelain', 'pvc', 'resin', 'rock wool', 'rubber', 'rvs', 'sand', 'sintels', 'snail wool', 'snails', 'starch', 'steel', 'stones', 'stories', 'sugar', 'teer', 'turf', 'tyleen', 'water', 'wine', 'wood', 'zin

In [13]:
print(data)

TabularDataBunch;

Train: LabelList (1870 items)
x: TabularList
euralDescription material unsuitable for consumption or processing; description Soyadroes technically; euralCode 20304; ,euralDescription waste from forestry; description Branches; euralCode 20107; ,euralDescription material unsuitable for consumption or processing; description Cocoa shells; euralCode 20304; ,euralDescription mixtures of concrete, stones, tiles or ceramic products other than those mentioned in 17 01 06; description Debris with Sand; euralCode 170107; ,euralDescription material unsuitable for consumption or processing; description soya; euralCode 20304; 
y: CategoryList
organic material,wood,cocoa,sand,organic material
Path: /content/drive/My Drive/data/minimizedData.csv;

Valid: LabelList (244 items)
x: TabularList
euralDescription soil and stones not covered by 17 05 03; description #na#; euralCode 170504; ,euralDescription biodegradable kitchen and canteen waste; description #na#; euralCode 200108; ,eura

pass in data to tabular learner:


In [0]:
learn = tabular_learner(data, layers=[300,200], metrics=accuracy)

In [25]:
learn.fit(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.957855,2.474927,0.602459,00:00
1,1.142252,1.338708,0.680328,00:00
2,0.673341,1.379278,0.704918,00:00


In [26]:
# see into the model
# learn.model.eval()
learn.summary()


TabularModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [15]                 855        True      
______________________________________________________________________
Embedding            [104]                179,088    True      
______________________________________________________________________
Embedding            [18]                 1,404      True      
______________________________________________________________________
Dropout              [137]                0          False     
______________________________________________________________________
Linear               [300]                41,400     True      
______________________________________________________________________
ReLU                 [300]                0          False     
______________________________________________________________________
BatchNorm1d          [300]                600        True      
_________________________________________________

In [0]:
learn.model_dir = '/content/drive/My Drive/data/models'
learn.save('material_classifier_model')

## Inference  
- run test data set to predict material
- extract the prediction confindece score  

In [34]:
row = 129

# data_frame = df
data_frame = test_df

print('given material -->', data_frame.iloc[row].material)
print('description ---> ', data_frame.iloc[row].description, ', ', data_frame.iloc[row].euralDescription)


given material --> nan
description --->  9401 third party product ,  mixed municipal waste


In [35]:
#one_row = df.iloc[row]
one_row = test_df.iloc[row]

predicted_cat, total_cat, weights = learn.predict(one_row)
print('predicted material ----> ', predicted_cat)

predicted material ---->  organic material


Category Probability - Prediction Confidence score

In [0]:
cat, idx, confidence = learn.predict(one_row)
cat_prob = confidence[idx.item()].item()
print(' category probability --------->', cat_prob)
print(confidence.sum())

 category probability ---------> 0.21948103606700897
tensor(1.0000)
